In [1]:
import cv2                 # working with, mainly resizing, images
import numpy as np         # dealing with arrays
import os                  # dealing with directories
from random import shuffle # mixing up or currently ordered data that might lead our network astray in training.
from tqdm import tqdm      # a nice pretty percentage bar for tasks.
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import DBSCAN

In [2]:
MASK_DIR = "/home/sam/dissertation/mask_image"
DATA_DIR="/home/sam/dissertation/fish_image"
IMG_SIZE = 100
LR = 1e-3

MODEL_NAME = 'fishCNN_oct_parallel-{}-{}.model'.format(LR, '2conv-basic')

def createData():
    data=[]
#     mask=[]
    for dir in tqdm(os.listdir(DATA_DIR)):
#         mask_path = os.path.join(MASK_DIR,dir)
        img_path=os.path.join(DATA_DIR,dir)
        label=dir.split('_')[-1];
        oneHotLabel=np.zeros(23);
        oneHotLabel[int(label)-1]=1;
        for img in tqdm(os.listdir(img_path)):
            path = os.path.join(img_path,img)
            imgData = cv2.imread(path)
            imgData = cv2.resize(imgData, (IMG_SIZE,IMG_SIZE))
            
            
#             mpath = os.path.join(mask_path,img)
#             mpath=mpath.replace("fish","mask")
#             mask= cv2.imread(mpath,cv2.IMREAD_GRAYSCALE)
            
#             print mpath
#             print mask.shape
#             mask=cv2.resize(mask,(IMG_SIZE,IMG_SIZE))
            
#             imgData=np.dstack((imgData,mask))
            
            data.append([np.array(imgData),np.array(oneHotLabel)])
            
    shuffle(data)
    np.save('data_masked_new.npy', data)
    return data

In [14]:
NEW_DIR="/home/sam/dissertation/output/new_img"
IMG_SIZE = 100
LR = 1e-3

MODEL_NAME = 'fishCNN_oct_parallel-{}-{}.model'.format(LR, '2conv-basic')

def createData2(pointer):
    new_data=[]
    count = 0
    
    for img in tqdm(os.listdir(NEW_DIR)):
        path = os.path.join(NEW_DIR,img)
        imgData = cv2.imread(path)
        imgData = cv2.resize(imgData, (IMG_SIZE,IMG_SIZE))            
        new_data.append(np.array(imgData))
        count += 1
        if (count==10000):
            break
            
    shuffle(new_data)
    np.save('data_unknown.npy', new_data)
    return new_data

In [23]:
def partition_unknown():
    new_dir = "/home/sam/dissertation/output/new_img"
    IMG_SIZE = 100
    new_data = []
    count = 0
    total = 272725
    idx = 0
    marker = 10000
    
    for img in tqdm(os.listdir(new_dir)):
        path = os.path.join(new_dir, img)
        imgData = cv2.imread(path)
        imgData = cv2.resize(imgData, (IMG_SIZE,IMG_SIZE))            
        new_data.append(np.array(imgData))
        count += 1
        if (count==marker):
            marker += 10000
            shuffle(new_data)
            npy_name = "cnn_partitions/partition_" + str(idx) + ".npy"
            idx += 1
            np.save(npy_name, new_data)
            print("Saved {}, length = {}".format(npy_name, len(new_data)))
            new_data = []

In [5]:
data = createData()

 95%|█████████▍| 3845/4049 [00:26<00:01, 126.28it/s]


100%|██████████| 21/21 [00:00<00:00, 487.81it/s]

100%|██████████| 16/16 [00:00<00:00, 541.47it/s]

100%|██████████| 25/25 [00:00<00:00, 670.30it/s]

 17%|█▋        | 2084/12112 [00:23<01:47, 93.39it/s]


 42%|████▏     | 5064/12112 [00:56<01:22, 85.16it/s]


 66%|██████▌   | 7950/12112 [01:32<00:52, 79.62it/s]


 90%|█████████ | 10948/12112 [02:06<00:16, 68.51it/s]


 75%|███████▍  | 2689/3593 [00:19<00:06, 139.95it/s]


100%|██████████| 23/23 [03:45<00:00,  9.78s/it]


In [ ]:
data = np.load("data_new_new.npy", allow_pickle=True)

In [7]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

input_layer = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input')

convnet = conv_2d(input_layer, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
mp1 = max_pool_2d(convnet, 5)

fc1 = fully_connected(mp1, 1024, activation='relu')
do1 = dropout(fc1, 0.8)


fc2 = fully_connected(do1, 23, activation='softmax')
do2 = regression(fc2, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(do2, tensorboard_dir='log_mask')

/home/sam/anaconda3/envs/mlp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sam/anaconda3/envs/mlp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sam/anaconda3/envs/mlp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sam/anaconda3/envs/mlp/lib/python3.7/site-packages/tensorf

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.


In [8]:
train = data[:-int(len(data)/20)]
test = data[-int(len(data)/20):]

In [9]:
X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
test_y = [i[1] for i in test]

#X.dtype

# X=X.view('float32')
# test_x/=test_x.view('float32')

# test_xY[0].shape

INFO:tensorflow:/home/sam/dissertation/fishCNN_oct_parallel-0.001-2conv-basic.model is not in all_model_checkpoint_paths. Manually adding it.


FailedPreconditionError: Attempting to use uninitialized value Accuracy/Mean/moving_avg_1
	 [[node save_4/SaveV2 (defined at /home/sam/anaconda3/envs/mlp/lib/python3.7/site-packages/tflearn/helpers/trainer.py:137) ]]

Caused by op 'save_4/SaveV2', defined at:
  File "/home/sam/anaconda3/envs/mlp/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/sam/anaconda3/envs/mlp/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/sam/anaconda3/envs/mlp/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/sam/anaconda3/envs/mlp/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/home/sam/anaconda3/envs/mlp/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 583, in start
    self.io_loop.start()
  File "/home/sam/anaconda3/envs/mlp/lib/python3.7/site-packages/tornado/platform/asyncio

In [12]:
model.fit({'input': X}, {'targets': Y}, n_epoch=3, snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

Training Step: 34  | total loss: 1.82215 | time: 4.010s
| Adam | epoch: 003 | loss: 1.82215 - acc: 0.4260 -- iter: 00384/26002


KeyboardInterrupt: 

In [10]:
new_model = tflearn.DNN(mp1, session=model.session)

In [30]:
# data_0 = np.load("cnn_partitions/partition_0.npy", allow_pickle=True)
unknown_preds = new_model.predict(data_0)

ResourceExhaustedError: OOM when allocating tensor with shape[10000,100,100,32] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node Conv2D/Relu (defined at /home/sam/anaconda3/envs/mlp/lib/python3.7/site-packages/tflearn/activations.py:131) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'Conv2D/Relu', defined at:
  File "/home/sam/anaconda3/envs/mlp/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/sam/anaconda3/envs/mlp/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/sam/anaconda3/envs/mlp/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/sam/anaconda3/envs/mlp/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File 

In [31]:
print(unknown_preds.shape)

NameError: name 'unknown_preds' is not defined

In [ ]:
columns = list(range(64))
df = pd.DataFrame(columns=columns)
for i, item in enumerate(orig_preds):
    df.loc[i] = item

In [ ]:
np.dstack(test_x[5,:,:,np.array([0,1,2])]).shape

In [ ]:
X.shape

In [ ]:
epss = [0.1,0.2,0.3,0.5,0.7,1]
svs = [2,3,4,5,6,8,10]
clustering_labels = []
for eps in epss:
    for sv in svs:
        clustering = DBSCAN(eps=eps, min_samples=sv).fit(df)
        clustering_labels.append(clustering.labels_)

In [ ]:
for i in range(len(clustering_labels)):
    print(clustering_labels[i])
    print()

In [ ]:
n = 40
plt.imshow(np.dstack(test_x[n,:,:,np.array([2,1,0])]))
plt.show()

print (model.predict([test_x[n]]))
print (model.predict_label([test_x[n]]))

In [ ]:
print (model.predict([test_x[n]]))

In [ ]:
np.save("new_data.npy", data)

In [ ]:
idx = 10000
for i in range(25):
    partition = 